# INF8460: Traitement automatique de la langue naturelle

### TP2: Autocomplétion et génération de phrases avec des modèles de langue n-grammes

## Identification de l'équipe:

### Groupe de laboratoire:

### Equipe numéro :

### Membres:

- Lucas Bertinchamp (2312324) (% de contribution, nature de la contribution)
- Antoine Toussaint (2312379) (% de contribution, nature de la contribution)
- Sebastian Villanueva (2087346) (% de contribution, nature de la contribution)

* nature de la contribution: Décrivez brièvement ce qui a été fait par chaque membre de l’équipe. Tous les membres sont censés contribuer au développement. Bien que chaque membre puisse effectuer différentes tâches, vous devez vous efforcer d’obtenir une répartition égale du travail.

TODOOOOOO : Q 2.4.2, valeurs différentes de ce qui est attendu. Random seed des jeux de train et test ?  

## Description:

Ce deuxième travail pratique sera dédié à l'utilisation des n-grammes en traitement du langage naturel (NLP). Les n-grammes sont des séquences de n mots consécutifs dans un texte, et ils sont présent dans de nombreuses applications du NLP.

Au cours de ce laboratoire, nous allons explorer comment les n-grammes sont utilisés pour des tâches telles que la prédiction de mots, l'autocomplétion et la génération de texte. Voici quelques exemples concrets de leur utilisation :

Prédiction de Mots : Les n-grammes sont utilisés pour prédire le mot suivant étant donné le contexte précédent. Par exemple, dans la phrase "Je pense donc je ____", un modèle de bigramme pourrait suggérer "suis" comme mot suivant, basé sur des observations antérieures.

Autocomplétion : Les moteurs de recherche utilisent les n-grammes pour proposer des suggestions de recherche lorsque vous commencez à taper une phrase. Ils se basent sur les préfixes du mot courant et sur les phrases écrites par l'utilisateur.

Génération de Texte : Les n-grammes peuvent être utilisés pour générer automatiquement du texte. Ils aident un modèle à prédire les mots suivants en fonction des probabilités des mots pouvant être générés selon le contexte.

Vous étudierez aussi une manière d'évaluer la qualité des modèles de langage génératifs en utilisant la perplexité. La perplexité est une métrique qui évalue la confiance avec laquelle un modèle est capable de prédire une séquence de mots dans un texte. Plus la perplexité est faible, plus le modèle est capable de prédire avec précision. Autrement dit, la perplexité est une mesure de la "surprise" d'un modèle lorsqu'il est exposé à un nouvel ensemble de mots. Nous utiliserons cette métrique pour évaluer nos modèles basés sur les n-grammes.


**NOTE: seulement les librairies standards de python (et numpy) sont permises ainsi que celles déjà importées le notebook**

<a name='1'></a>
## 1.  Chargement et pré-traitement des données (12 points)

<a name='1.1'></a>
### 1.1 Chargement des données (1 point)

Les données que vous allez utiliser dans ce travail sont contenues dans le fichier [trump.txt](./trump.txt)

Lisez le contenu de ce fichier et stockez-le dans une variable data.

Affichez ensuite les 300 premiers caractères



In [2]:
filename = "trump.txt"
file = open(filename, 'r')
data = file.read()

data[:300]


'Thank you very much.\nWe had an amazing convention.\nThat was one of the best.\nI think it was one of the best ever.\nIn terms -- in terms of enthusiasm, in terms of I think what it represents, getting our word out.\nIvanka was incredible last night.\nShe did an incredible job.\nAnd so many of the speakers'

<a name='1.2'></a>
### 1.2  Segmentation (2 points)

Pré-traitez les données en suivant les étapes suivantes:

1. Enlever les majuscules.
2. Remplacer les "\n" par des espaces
3. Séparer les données en phrases en utilisant les délimiteurs suivants `.`, `?` et `!` comme séparateur.
4. Enlever les signes de ponctuation (Attention de garder les espaces).
5. Enlever les phrases vides.
6. Segmenter les phrases avec la fonction nltk.word_tokenize()

Utilisez ensuite votre fonction pour prétraiter le jeu de données.

In [3]:
import nltk
import re

def preprocess(data):
    # Elever les majuscules
    data_processed = data.lower()

    # Remplacer les "\n" par des espaces
    data_processed = data_processed.replace('\n', ' ')

    # Séparer les données en phrases en utilisant les délimiteurs suivants `.`, `?` et `!` comme séparateur.
    data_processed = re.split(r'[.?!]', data_processed)

    # Enlever les signes de ponctuation (Attention de garder les espaces).
    data_processed = [re.sub(r'[^\w\s]', '', sentence) for sentence in data_processed]

    # Enlever les phrases vides.
    data_processed = [sentence for sentence in data_processed if sentence != '']

    # Segmenter les phrases avec la fonction nltk.word_tokenize()
    data_processed = [nltk.word_tokenize(sentence) for sentence in data_processed]

    return data_processed



In [4]:
data = preprocess(data)

In [5]:
# test
x = "Cats are independent.\nDogs are faithful."
preprocess(x)

[['cats', 'are', 'independent'], ['dogs', 'are', 'faithful']]

##### Sortie attendue

```CPP
[['cats', 'are', 'independent'], ['dogs', 'are', 'faithful']]
```

<a name='1.3'></a>
###  1.3 Création d'ensembles d'entraînement et de test (1 point)

Échantilloner de manière **aléatoire** 80% des données pour l'ensemble d'entrainement. Garder 20% pour l'ensemble de test. Utilisez la fonction train_test_split de sklearn. Stocker les résultats dans des variables.

In [6]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, random_state=42)


<a name='1.4'></a>
### 1.4 Construction du vocabulaire (4 points)

Comme dans le TP1, construisez un vocabulaire à partir des données d'entraînement. Vous pouvez reprendre votre code du TP1.

Complétez la fonction **build_voc** qui retourne une liste de jetons qui sont présents au moins n fois (threshold passé en paramètre) dans la liste d'exemples (également passée en paramètre). Vous pouvez utiliser la classe collections.Counter.

Ensuite, appelez cette fonction pour construire votre vocabulaire à partir de l'ensemble d'entraînement **en utilisant threshold=2**. Imprimez la taille du vocabulaire.


In [7]:
from collections import Counter

def build_voc(documents, threshold):

    word_counts = Counter()

    for sentence in documents:
        for word in sentence:
            word_counts[word] += 1

    vocabulary = [word for word, count in word_counts.items() if count >= threshold]

    return vocabulary

In [8]:
voc = build_voc(train, 2)
print("Vocabulary size:", len(voc))

Vocabulary size: 5608


<a name='1.5'></a>
### 1.5 Mots hors vocabulaire (4 points)

Si votre modèle réalise de l'autocomplétion, mais qu'il rencontre un mot qu'il n'a jamais vu lors de l'entraînement, le modèle ne pourra donc pas prédire le mot suivant car il n'y a pas d'occurrence pour le mot actuel.

Ces mots sont appelés les mots hors vocabulaire (Out of Vocabulary) <b>OOV</b>.
Le pourcentage de mots inconnus dans l'ensemble de test est appelé le taux de mots <b> OOV </b>.

Pour gérer les mots inconnus lors de la prédiction, utilisez un jeton spécial 'unk' pour représenter tous les mots inconnus. Plus spécifiquement, la technique que vous utiliserez sera la suivante:

Complétez la fonction replace_oov qui convertit tous les mots qui ne font pas partie du vocabulaire en jeton '\<unk\>'.

Appelez ensuite votre fonction sur votre corpus d'entraînement et de test.



In [9]:
def replace_oov(tokenized_sentences, voc):

    tokenized_sentences_processed = []

    for sentence in tokenized_sentences:
        sentence_processed = []
        for word in sentence:
            sentence_processed.append(word) if word in voc else sentence_processed.append('<unk>')
        tokenized_sentences_processed.append(sentence_processed)

    return tokenized_sentences_processed


In [10]:
train = replace_oov(train, voc)
test = replace_oov(test, voc)

In [11]:
tokenized_sentences = [["cats", "sleep"], ["mice", "eat"], ["cats", "and", "mice"]]
vocabulary = build_voc([["cats", "sleep"], ["mice", "eat"], ["cats", "and", "mice"]], 2)
tmp_replaced_tokenized_sentences = replace_oov(tokenized_sentences, vocabulary)
print(f"Phrase initiale:")
print(tokenized_sentences)
print(f"Phrase segmentée avec'<unk>':")
print(tmp_replaced_tokenized_sentences)

Phrase initiale:
[['cats', 'sleep'], ['mice', 'eat'], ['cats', 'and', 'mice']]
Phrase segmentée avec'<unk>':
[['cats', '<unk>'], ['mice', '<unk>'], ['cats', '<unk>', 'mice']]


##### Sortie attendue
```CPP
Phrase initiale:
[['cats', 'sleep'], ['mice', 'eat'], ['cats', 'and', 'mice']]
Phrase segmentée avec '<unk>':
[['cats', '<unk>'], ['mice', '<unk>'], ['cats', '<unk>', 'mice']]
```

<a name='2'></a>
## 2. Modèles de langue n-gramme (18 points)



Dans cette section, vous développerez un modèle de langue n-grammes. Nous allons utiliser la formule:

$$ \hat{P}(w_t | w_{t-1}\dots w_{t-n}) = \frac{C(w_{t-1}\dots w_{t-n}, w_t)}{C(w_{t-1}\dots w_{t-n})} \tag{2} $$

- La fonction $C(\cdots)$ représente le nombre d'occurrences de la séquence donnée.
- $\hat{P}$ signifie l'estimation de $P$.

Vous pouvez estimer cette probabilité en comptant les occurrences de ces séquences de mots dans les données d'entraînement.


<a name='2.1'></a>
### 2.1 Fréquence des n-grammes (3 points)



Vous allez commencer par mettre en œuvre une fonction qui calcule la fréquence des n-grammes pour un nombre arbitraire $n$.

Vous devez pré-traiter la phrase en ajoutant $n$ marqueurs de début de phrase "\<s\>" pour indiquer le commencement de la phrase.

- Par exemple, dans un modèle bigramme (N=2), la séquence devrait commencer avec deux jetons de début "\<s\>\<s\>". Ainsi, si la phrase est "J'aime la nourriture", modifiez-la pour devenir "\<s\>\<s\> J'aime la nourriture".
- Ajoutez aussi un jeton de fin "\<e\>" pour que le modèle puisse prédire quand terminer une phrase.
    

Dans cette implémentation, vous devez stocker les occurrences des n-grammes sous forme de dictionnaire.

- La clé de chaque paire clé-valeur dans le dictionnaire est un tuple de n mots (et non une liste).
- La valeur dans la paire clé-valeur est le nombre d'occurrences.

In [12]:

def count_n_grams(data, n, start_token='<s>', end_token = '<e>'):

    processed_data = []

    # Process the data
    for sentence in data:
        sentence = [start_token] * n + sentence + [end_token]
        processed_data.append(sentence)

    # Count the number of occurence of each n-gram
    n_grams = {}
    for sentence in processed_data:
        for i in range(len(sentence)-n+1):
            n_gram = tuple(sentence[i:i+n])
            if n_gram in n_grams:
                n_grams[n_gram] += 1
            else:
                n_grams[n_gram] = 1

    return n_grams

In [13]:
sentences = [['i', 'have', 'a', 'mouse'],
             ['this', 'mouse', 'likes', 'cats']]
print("Unigrammes:")
print(count_n_grams(sentences, 1))
print("Bigrammes:")
print(count_n_grams(sentences, 2))

Unigrammes:
{('<s>',): 2, ('i',): 1, ('have',): 1, ('a',): 1, ('mouse',): 2, ('<e>',): 2, ('this',): 1, ('likes',): 1, ('cats',): 1}
Bigrammes:
{('<s>', '<s>'): 2, ('<s>', 'i'): 1, ('i', 'have'): 1, ('have', 'a'): 1, ('a', 'mouse'): 1, ('mouse', '<e>'): 1, ('<s>', 'this'): 1, ('this', 'mouse'): 1, ('mouse', 'likes'): 1, ('likes', 'cats'): 1, ('cats', '<e>'): 1}


Sortie attendue:

```CPP
Unigrammes:
{('<s>',): 2, ('i',): 1, ('have',): 1, ('a',): 1, ('mouse',): 2, ('<e>',): 2, ('this',): 1, ('likes',): 1, ('cats',): 1}
Bigrammes:
{('<s>', '<s>'): 2, ('<s>', 'i'): 1, ('i', 'have'): 1, ('have', 'a'): 1, ('a', 'mouse'): 1, ('mouse', '<e>'): 1, ('<s>', 'this'): 1, ('this', 'mouse'): 1, ('mouse', 'likes'): 1, ('likes', 'cats'): 1, ('cats', '<e>'): 1}

```

<a name='2.2'></a>
### 2.2 Estimé du maximum de vraisemblance MLE (4 points)

#### 2.2.1 Calcul de probabilité pour un mot (3 points)


Ensuite, estimez la probabilité d'un mot étant donnés les 'n' mots précédents avec les fréquences obtenues.

$$ \hat{P}(w_t | w_{t-1}\dots w_{t-n}) = \frac{C(w_{t-1}\dots w_{t-n}, w_t)}{C(w_{t-1}\dots w_{t-n})} \tag{2}$$


La fonction prend en entrée:

- word : le mot dont on veut estimer la probabilité
- previous_n_gram : le n-gramme précédent, sous forme de tuple
- n_gram_counts: Un dictionnaire où la clé est le n-gramme et la valeur est la fréquence de ce n-gramme.
- n_plus1_gram_counts: Un autre dictionnaire, que vous utiliserez pour trouver la fréquence du n-gramme précédent plus le mot actuel.


In [44]:
def estimate_probability(word, previous_n_gram, n_gram_counts, n_plus1_gram_counts):

    # On adapte la taille du previous_n_gram en fonction de la taille de n_gram_counts
    len_context = len(next(iter(n_gram_counts)))
    if len(previous_n_gram) > len_context:
        previous_n_gram = previous_n_gram[-len_context:]
    elif len(previous_n_gram) < len_context:
        previous_n_gram = (len_context - len(previous_n_gram)) * ('<s>',) + previous_n_gram

    previous_n_gram_count = n_gram_counts[previous_n_gram] if previous_n_gram in n_gram_counts else 0

    n_plus_1_gram = previous_n_gram + (word,)
    n_plus_1_gram_count = n_plus1_gram_counts[n_plus_1_gram] if n_plus_1_gram in n_plus1_gram_counts else 0

    probability = n_plus_1_gram_count / previous_n_gram_count

    return probability

#### 2.2.1 Quel est le problème de cette fonction? Quelle embûche pourrait-on rencontrer? Répondre avec un exemple. (1 points)

> Si on cherche la probabilité d'un mot sachant un n-gramme qui n'appartient pas à notre dictionnaire, alors le dénominateur sera égal à 0. On ne pourra donc pas calculer la probabilité du mot. (Exemple ci-dessous)

In [45]:
prob = estimate_probability(word='mouse', previous_n_gram=('<s>', 'I'), n_gram_counts=count_n_grams(sentences, 2), n_plus1_gram_counts=count_n_grams(sentences, 3))
print(prob)

ZeroDivisionError: division by zero

<a name='2.3'></a>
### 2.3  Lissage add-k (4 points)

Vous allez maintenant modifier votre fonction précédente en utilisant le lissage add-k.

$$ \hat{P}(w_t | w_{t-1}\dots w_{t-n}) = \frac{C(w_{t-1}\dots w_{t-n}, w_n) + k}{C(w_{t-1}\dots w_{t-n}) + k|V|} \tag{3} $$

Recodez la fonction au numéro 2.2 en ajoutant une constante de lissage $k$ and la taille du vocabulaire en paramètres supplémentaires.

In [46]:
def estimate_probability_smoothing(word, previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):

    # On adapte la taille du previous_n_gram en fonction de la taille de n_gram_counts
    len_context = len(next(iter(n_gram_counts)))
    if len(previous_n_gram) > len_context:
        previous_n_gram = previous_n_gram[-len_context:]
    elif len(previous_n_gram) < len_context:
        previous_n_gram = (len_context - len(previous_n_gram)) * ('<s>',) + previous_n_gram

    previous_n_gram_count = n_gram_counts[previous_n_gram] if previous_n_gram in n_gram_counts else 0

    n_plus_1_gram = previous_n_gram + (word,)
    n_plus_1_gram_count = n_plus1_gram_counts[n_plus_1_gram] if n_plus_1_gram in n_plus1_gram_counts else 0

    probability = (n_plus_1_gram_count + k) / (previous_n_gram_count + k*vocabulary_size)

    return probability

In [47]:
# test
sentences = [['i', 'have', 'a', 'mouse'],
             ['this', 'mouse', 'likes', 'cats']]
unique_words = list(set(sentences[0] + sentences[1]))

bigram_counts = count_n_grams(sentences, 2)
trigram_counts = count_n_grams(sentences, 3)
tmp_prob = estimate_probability_smoothing("have", ('<s>', 'i',), bigram_counts, trigram_counts, len(unique_words), k=1)

print(f" La probabilité de 'have' étant donné le mot précédent 'i' est: {tmp_prob:.4f}")

 La probabilité de 'have' étant donné le mot précédent 'i' est: 0.2500


##### Sortie attendue

```CPP
 La probabilité de 'have' étant donné le mot précédent 'i' est: 0.2500
```

<a name='2.4'></a>
### 2.4 Calcul des probabilités des n-grammes (7 points)

#### 2.4.1. Estimation des probabilités (4 points)
Complétez la fonction estimate_probabilities qui calcule pour chaque mot du vocabulaire la probabilité d'être généré en utilisant la fonction avec lissage add-k.

N'oubliez pas d'ajouter le jetons spécial "\<e\>" au vocabulaire

Cette fonction prends en entrée:
- previous_n_gram: le n-gramme précédent, sous forme de tuple
- n_gram_counts: Un dictionnaire où la clé est le n-gramme et la valeur est la fréquence de ce n-gramme.
- n_plus1_gram_counts: Un autre dictionnaire, que vous utiliserez pour trouver la fréquence du n-gramme précédent plus le mot actuel.
- vocabulary: le vocabulaire
- k: la constante de lissage

La fonction retourne un dictionnaire ayant pour clés tous les mots du vocabulaire ainsi que leur probabilité d'être générés

In [48]:
def estimate_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0):

    # On ajoute le token de fin de phrase s'il n'est pas déjà présent ou a été oublié
    vocabulary = set(vocabulary).union(set(['<e>']))

    words_probabilities = {}
    for word in vocabulary:
        words_probabilities[word] = estimate_probability_smoothing(word, previous_n_gram, n_gram_counts, n_plus1_gram_counts, len(vocabulary), k)

    return words_probabilities


In [49]:
# test
sentences = [['i', 'have', 'a', 'mouse'],
             ['this', 'mouse', 'likes', 'cats']]
unique_words = list(set(sentences[0] + sentences[1]))
unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
estimate_probabilities(("a",), unigram_counts, bigram_counts, unique_words, k=1)

{'cats': 0.1111111111111111,
 'mouse': 0.2222222222222222,
 'this': 0.1111111111111111,
 '<e>': 0.1111111111111111,
 'i': 0.1111111111111111,
 'likes': 0.1111111111111111,
 'have': 0.1111111111111111,
 'a': 0.1111111111111111}

##### Sortie attendue

```CPP
{'likes': 0.1111111111111111,
 'have': 0.1111111111111111,
 'this': 0.1111111111111111,
 'i': 0.1111111111111111,
 'mouse': 0.2222222222222222,
 'a': 0.1111111111111111,
 'cats': 0.1111111111111111,
 '<e>': 0.1111111111111111}
```

#### 2.4.2. Probabilités étant donné un contexte (3 points)

Affichez maintenant les probabilités des tri-grammes étant donné le context "i will" en utilisant les données d'entraînement . N'affichez que les 10 mots les plus probables en ordre décroissant de probabilité. Utilisez K=1

In [123]:
unigram_counts = count_n_grams(train, 1)
bigram_counts = count_n_grams(train, 2)
trigram_counts = count_n_grams(train, 3)

voc = build_voc(train, 2)
voc.append('<e>')

probabilities = estimate_probabilities(("i", "will",), bigram_counts, trigram_counts , voc, k=1)
probabilities_sorted = sorted(probabilities.items(), key=lambda x: x[1], reverse=True)

print(probabilities_sorted[:10])

[('tell', 0.008265856950067476), ('fight', 0.005566801619433198), ('fix', 0.0052294197031039135), ('be', 0.004554655870445344), ('never', 0.004048582995951417), ('say', 0.00354251012145749), ('not', 0.0021929824561403508), ('ask', 0.0020242914979757085), ('work', 0.0016869095816464238), ('also', 0.0016869095816464238)]


##### Sortie attendue

```CPP
[('tell', 0.0070981916511745815),
 ('fix', 0.005239141456819334),
 ('fight', 0.005239141456819334),
 ('be', 0.0050701368936961295),
 ('never', 0.004225114078080108),
 ('say', 0.003718100388710495),
 ('not', 0.0025350684468480648),
 ('ask', 0.0023660638837248605),
 ('also', 0.0018590501943552475),
 ('work', 0.001521041068108839)]
```

<a name='3'></a>
## 3. Perplexité (15 points)

Dans cette section, vous allez générer le score de perplexité pour évaluer votre modèle sur l'ensemble de test.

Pour calculer le score de perplexité d'une phrase sur un modèle n-gramme, utilisez :

$$PP(W) =\sqrt[N]{ \prod_{t=1}^{N} \frac{1}{P(w_t | w_{t-n} \cdots w_{t-1})} } \tag{4.1}$$

où N = le nombre de jeton dans la phrases incluant le jeton \<e\>
et P = la probabilité de générer le jeton $w_t$

Plus les probabilités sont élevées, plus la perplexité sera basse.

<a name='3.1'></a>
### 3.1. Calcul de la perplexité (4 points)
Complétez la fonction `calculate_perplexity`, qui pour une phrase donnée, nous donne le score de perplexité. Cette fonction prend en entrée:


- sentence: La phrase pour laquelle vous devez calculer la perplexité
- n_gram_counts: Un dictionnaire où la clé est le n-gramme et la valeur est la fréquence de ce n-gramme.
- n_plus1_gram_counts: Un autre dictionnaire, que vous utiliserez pour trouver la fréquence du n-gramme précédent plus le mot actuel.
- vocabulary_size: la taille du vocabulaire
- k: la constante de lissage

In [51]:
def calculate_perplexity(sentence, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):

    # Le nombre de jetons (on ajoute 1 pour le token de fin de phrase)
    N = len(sentence) + 1
    # On récupère la taille du n-gramme
    n = len(list(n_gram_counts.keys())[0])

    proba_total = 1

    sentence = ["<s>"] * n + sentence + ["<e>"]
    for i in range(n, N):
        n_gram = tuple(sentence[i-n:i])
        word = sentence[i]
        proba_total *= 1 / estimate_probability_smoothing(word, n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k)

    perplexity = proba_total**(1/N)

    return perplexity


In [52]:
# test

sentences = [['i', 'have', 'a', 'mouse'],
             ['this', 'mouse', 'likes', 'cats']]
unique_words = list(set(sentences[0] + sentences[1] + ['<e>']))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)


perplexity = calculate_perplexity(sentences[0],
                                         unigram_counts, bigram_counts,
                                         len(unique_words), k=1.0)
print(f"Perplexité de la première phrase: {perplexity:.4f}")


Perplexité de la première phrase: 3.4019


<a name='3.2'></a>
### 3.2. Perplexité sur une phrase d'entraînement (4 points)
Calculez et affichez la perplexité des modèles bi-grammes, tri-grammes et quadri-grammes à l'aide de votre fonction `calculate_perplexity` définie plus haut sur la première phrase de votre corpus d'entraînement. Utilisez K=0.01 ici.

In [53]:
bigram_counts = count_n_grams(train, 2)
trigram_counts = count_n_grams(train, 3)
quadrigram_counts = count_n_grams(train, 4)
quintigram_counts = count_n_grams(train, 5)

print(f"Perplexité du modèle bi-gramme :")
print(calculate_perplexity(train[0], bigram_counts, trigram_counts, len(voc), k=0.01))

print(f"Perplexité du modèle tri-gramme :")
print(calculate_perplexity(train[0], trigram_counts, quadrigram_counts, len(voc), k=0.01))

print(f"Perplexité du modèle quadri-gramme :")
print(calculate_perplexity(train[0], quadrigram_counts, quintigram_counts, len(voc), k=0.01))

Perplexité du modèle bi-gramme :
23.159151227367058
Perplexité du modèle tri-gramme :
20.336762114542754
Perplexité du modèle quadri-gramme :
17.122946540155546


<a name='3.3'></a>
### 3.3. Perplexité du corpus de test (7 points)

#### 3.3.1. Vous pouvez maintenant calculer et afficher la perplexité des modèles bi-grammes, tri-grammes et quadri-grammes sur votre corpus de test. K=1 ici. (4 points)

Pour calculer la perplexité d'un corpus de *m* phrases, il suffit de suivre la formule suivante :

Soit $N$ le nombre total de jetons dans le corpus de test C et $N_i$ le nombre de jetons dans la phrase i.

$$Perplexity(C) = \Big(\frac{1}{P(s_1, ..., s_m)}\Big)^{1/N}$$
$$P(s_1, ..., s_m) = \prod_{i=1}^{m} p(s_i)$$
$$p(s_i) = \prod_{t=1}^{N_i} \hat{P}(w_t | w_{t-n} \cdots w_{t-1})$$

Puisqu'il s'agit d'un multiplication de probabilités (situées entre 0 et 1), le produit devient nul très rapidement. C'est pourquoi il est plus efficace d'effectuer une transformation vers un espace logarithmique pour transformer les multiplications en addition. Cela donne ainsi la formule suivante:

$$LogPerplexity(C) = 2^{-\frac{1}{N} \sum_{k=1}^{m} log_{2} \; p(s_k)}$$


In [54]:
import numpy as np
def calculate_perplexity_corpus(corpus, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):

    # n est la taille du n-gramme
    n = len(list(n_gram_counts.keys())[0])

    # Compter les jetons du corpus
    N = 0
    for sentence in corpus:
        N += len(sentence) + 1 # +1 pour le token de fin de phrase

    somme_log = 0

    for sentence in corpus:
        sentence = ["<s>"] * n + sentence + ["<e>"]
        p_s = 1
        for i in range(n, len(sentence)):
            n_gram = tuple(sentence[i-n:i])
            word = sentence[i]
            p_s *= estimate_probability_smoothing(word, n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k)
        somme_log += np.log2(p_s)

    log_perplexity = -somme_log/N

    return 2**log_perplexity


In [55]:
n_gram_counts = {('<s>', 'quick'): 1, ('the', 'quick'): 1, ('quick', 'brown'): 1, ('brown', 'fox'): 1, ('jumps', 'over'): 1, ('over', 'the'): 1, ('the', 'lazy'): 1, ('lazy', 'dog'): 1, ('dog', '<e>'): 1}
n_plus1_gram_counts = { ('<s>', '<s>', 'the', ): 1, ('<s>', 'the', 'quick'): 1, ('the', 'quick', 'brown'): 1, ('quick', 'brown', 'fox'): 1, ('jumps', 'over', 'the'): 1, ('over', 'the', 'lazy'): 1, ('the', 'lazy', 'dog'): 1, ('lazy', 'dog', '<e>'): 1}

train_corpus = [["the", "quick", "brown", "fox"], ["jumps", "over", "the", "lazy", "dog"]]
n_gram_counts = {('<s>', '<s>'): 2, ('<s>', 'the'): 1, ('<s>', 'jumps'): 1, ('the', 'quick'): 1, ('quick', 'brown'): 1, ('brown', 'fox'): 1, ('fox', '<e>'): 1, ('jumps', 'over'): 1, ('over', 'the'): 1, ('the', 'lazy'): 1, ('lazy', 'dog'): 1, ('dog', '<e>'): 1}
n_plus1_gram_counts = {('<s>', '<s>', '<s>', ): 2, ('<s>', '<s>', 'the', ): 1, ('<s>', 'the', 'quick'): 1,  ('<s>', '<s>', 'jumps', ): 1, ('<s>', 'jumps', 'over'): 1, ('the', 'quick', 'brown'): 1, ('quick', 'brown', 'fox'): 1, ('brown', 'fox', '<e>'): 1, ('jumps', 'over', 'the'): 1, ('over', 'the', 'lazy'): 1, ('the', 'lazy', 'dog'): 1, ('lazy', 'dog', '<e>'): 1}

vocabulary = ["the", "quick", "brown", "fox", "jumps", "over", "lazy", "dog", "<e>"]

N = 5
V = len(vocabulary)

test_corpus = [["the", "fox"], ["jumps"]]

# Complétez le calcul de la perplexité avec k=1

perplexity = calculate_perplexity_corpus(test_corpus, n_gram_counts, n_plus1_gram_counts, V, k=1)
print(f"Perplexité du corpus de test: {perplexity}")


Perplexité du corpus de test: 7.708920690856638


#### Sortie attendue

    Perplexité du corpus de test:  7.708920690856638

In [56]:
# Calculez mainenant la perplexité de votre corpus de test
bigram_counts = count_n_grams(train, 2)
trigram_counts = count_n_grams(train, 3)
quadrigram_counts = count_n_grams(train, 4)
quintgram_counts = count_n_grams(train, 5)

voc = build_voc(train, 2)
voc.append('<e>')

print(f"Perplexité du corpus de test avec le modèle bi-gramme :")
print(calculate_perplexity_corpus(test, bigram_counts, trigram_counts, len(voc), k=0.01))

print(f"Perplexité du corpus de test avec le modèle tri-gramme :")
print(calculate_perplexity_corpus(test, trigram_counts, quadrigram_counts, len(voc), k=0.01))

print(f"Perplexité du corpus de test avec le modèle quadri-gramme :")
print(calculate_perplexity_corpus(test, quadrigram_counts, quintgram_counts, len(voc), k=0.01))

Perplexité du corpus de test avec le modèle bi-gramme :
208.63191684235457
Perplexité du corpus de test avec le modèle tri-gramme :
490.425193857908
Perplexité du corpus de test avec le modèle quadri-gramme :
750.6077767312561


#### 3.3.2. Les perplexités attendues peuvent sembler contre-intuitives.  Comparez-les aux perplexités obtenues sur l'ensemble d'entrainement pour les mêmes modèles. Comment expliquez-vous ces résultats et quelle est votre conclusion ?  (3 points)

Sur le corpus d'entraînement :

In [57]:
print(f"Perplexité du corpus de test avec le modèle bi-gramme :")
print(calculate_perplexity_corpus(train, bigram_counts, trigram_counts, len(voc), k=0.01))

print(f"Perplexité du corpus de test avec le modèle tri-gramme :")
print(calculate_perplexity_corpus(train, trigram_counts, quadrigram_counts, len(voc), k=0.01))

print(f"Perplexité du corpus de test avec le modèle quadri-gramme :")
print(calculate_perplexity_corpus(train, quadrigram_counts, quintgram_counts, len(voc), k=0.01))

Perplexité du corpus de test avec le modèle bi-gramme :
33.04071284302768
Perplexité du corpus de test avec le modèle tri-gramme :
37.54620559913497
Perplexité du corpus de test avec le modèle quadri-gramme :
41.45555774085327


On remarque que la perplexité augmente avec la taille des n-grammes. Cela peut s'expliquer par le besoin croissant de taille de corpus avec la longueur des n-grammes : plus les n-grammes sont longs, plus les mots hors-vocabulaire seront courants. La perplexité se basant sur les probabilités des n-grammes, plus les n-grammes sont longs, et plus ces probabilités auront tendance à être faibles car auront moins été vues. Il n'est donc pas cohérent de comparer des perplexités de n-grammes de longueur différentes.

<a name='4'></a>
## 4. Construction d'un modèle d'auto-complétion (15 points)

Dans cette dernière partie, vous allez utiliser les modèles n-grammes construits aux numéros précédents afin de faire un modèle d'autocomplétion.

<a name='4.1'></a>
### 4.1 Suggestion d'un mot à partir d'un préfixe (5 points)


La première étape sera de construire une fonction qui suggère un mot à partir des premiers caractères entrés par un utilisateur, considérant un seul type de n-gramme.  

Complétez la fonction `suggest_word` qui calcule les probabilités pour tous les mots suivants possibles et suggère le mot le plus probable. Comme contrainte supplémentaire, le mot suggéré doit commencer avec le préfixe passé en paramètre. Utilisez vos fonctions provenant du numéro 2. (Modèle n-gramme de mots) pour faire vos prédictions.

Cette fonction prends en paramètre:
- previous_n_gram: le n-gramme précédent, sous forme de tuple
- n_gram_counts: Un dictionnaire où la clé est le n-gramme et la valeur est la fréquence de ce n-gramme.
- n_plus1_gram_counts: Un autre dictionnaire, que vous utiliserez pour trouver la fréquence du n-gramme précédent plus le mot actuel.
- vocabulary_size: la taille du vocabulaire
- k: la constante de lissage
- prefixe: Le début du mot que l'on veut prédire

Elle retourne le mot le plus probable avec la probabilité associée

In [58]:
def suggest_word(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0, prefixe=""):

    # On ajoute le token de fin de phrase s'il n'est pas déjà présent ou a été oublié
    vocabulary = set(vocabulary).union(set(['<e>']))

    # La gestion de la taille du contexte est déjà gérée dans estimate_probability_smoothing
    probabilities = estimate_probabilities(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k)

    # On trie les mots par probabilité décroissante
    probabilities_sorted = sorted(probabilities.items(), key=lambda x: x[1], reverse=True)

    # On récupère le mot le plus probable commençant par le préfixe
    for word, prob in probabilities_sorted:
        if word.startswith(prefixe):
            return word, prob

    return "", 0

In [59]:
# test
sentences = [['i', 'have', 'a', 'mouse'],
             ['this', 'mouse', 'likes', 'cats']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)

previous_tokens = ("i", "have",)
tmp_suggest1 = suggest_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0)
print(f" avec les mots précédents 'i have',\n\t le mot suggéré est `{tmp_suggest1[0]}` avec la probabilité {tmp_suggest1[1]:.4f}")

print()

tmp_starts_with = 'm'
tmp_suggest2 = suggest_word(previous_tokens, bigram_counts, trigram_counts, unique_words, k=1.0, prefixe=tmp_starts_with)
print(f"avec les mots précédents 'i have', et une suggestion qui commence par `{tmp_starts_with}`\n\t le mot suggéré est : `{tmp_suggest2[0]}` avec une probabilité de {tmp_suggest2[1]:.4f}")

 avec les mots précédents 'i have',
	 le mot suggéré est `a` avec la probabilité 0.2222

avec les mots précédents 'i have', et une suggestion qui commence par `m`
	 le mot suggéré est : `mouse` avec une probabilité de 0.1111


### Sortie attendue

```CPP
avec les mots précédents 'i have',
	 le mot suggéré est `a` avec la probabilité 0.2222

avec les mots précédents 'i have', et une suggestion qui commence par `m`
	 le mot suggéré est : `mouse` avec une probabilité de 0.1111


```

<a name='4.2'></a>
### 4.2 Suggestions multiples (5 points)

Afin de suggérer plusieurs mots à l'utilisateur, une stratégie que l'on peut utiliser est de retourner un ensemble de mots suggérés par plusieurs types de modèles n-grammes.

En utilisant la fonction `suggest_word` du numéro précédent, complétez la fonction `get_suggestions` qui retourne les suggestions des modèles n-grammes passés en paramètre. Vous devrez aussi enlever les doublons dans les suggestions s'il y en a, et ordonner la liste des suggestions en commençant par le mot ayant la probabilité la plus élevée.

La fonction get_suggestions prends en paramètres:
- previous_n_gram: le n-gramme précédent, sous forme de tuple
- n_gram_counts_list: une liste de n-grammes dans l'ordre suivant [unigrammes, bigrammes, trigrammes, quadrigrammes, ...]
- vocabulary_size: la taille du vocabulaire
- k: la constante de lissage (entre 0 et 1)
- prefixe: Le début du mot que l'on veut prédire, "" si au aucun préfixe

In [67]:
def get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, prefixe=""):

    suggested_word_list = []

    # La gestion de la taille du previous_tokens est faite dans la fonction estimate_probabilitiy_smoothing
    for i in range(len(n_gram_counts_list)-1):
        suggested_word_list.append(suggest_word(previous_tokens, n_gram_counts_list[i], n_gram_counts_list[i+1], vocabulary, k, prefixe))

    suggested_word_dict = {}

    # Enlève les doublons et garde la plus grande probabilité de chaque suggestion si un mot est suggéré par plusieurs modèles
    for word, prob in suggested_word_list:
        if word in suggested_word_dict:
            if prob > suggested_word_dict[word]:
                suggested_word_dict[word] = prob
        else:
            suggested_word_dict[word] = prob

    # Trie les suggestions par probabilité décroissante
    suggested_word = sorted(suggested_word_dict.items(), key=lambda x: x[1], reverse=True)

    # On ne garde que les mots
    suggested_word = [word for word, _ in suggested_word]

    return suggested_word

In [68]:
# test
sentences = [['i', 'have', 'a', 'mouse'],
             ['this', 'mouse', 'likes', 'cats']]
unique_words = list(set(sentences[0] + sentences[1] + ['<e>']))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
trigram_counts = count_n_grams(sentences, 3)
quadgram_counts = count_n_grams(sentences, 4)
qintgram_counts = count_n_grams(sentences, 5)

n_gram_counts_list = [unigram_counts, bigram_counts, trigram_counts, quadgram_counts, qintgram_counts]
previous_tokens = ("i", "have",)
tmp_suggest3 = get_suggestions(previous_tokens, n_gram_counts_list, unique_words, k=1.0)

print(f"Etant donné les mots i have, je suggère :")
display(tmp_suggest3)

Etant donné les mots i have, je suggère :


['a']

##### Sortie attendue

```CPP
Etant donné les mots i have, je suggère :
['a']
```

<a name='4.3'></a>
### 4.3 Autocomplétion (5 points)

Il est maintenant temps de combiner vos fonctions afin de créer le modèle d'autocomplétion. En utilisant le jeu de données d'entraînement, calculez la fréquence des n-grammes allant de 1 à 5 et utilisez la fonction *get_suggestions* afin de suggérer des mots. Vous devrez être en mesure de toujours suggérer des mots à partir du dernier mot entré par l'utilisateur.

Complétez la fonction *update_suggestions*:
- la variable texte_actuel contient tout le texte entré par l'utilisateur
- la variable top_suggestions contient les suggestions qui seront proposées

Vous devrez changer le contenu de la variable top_suggestions pour qu'elle contienne les suggestions des n-grammes.

In [33]:
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\Lucas\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [70]:
import ipywidgets as widgets
from IPython.display import display

text_input = widgets.Text(placeholder="Entrez votre text ici...")

suggestions_label = widgets.Label(value="Suggestions: ")

# Génère les n-grammes pour le modèle
unigram_counts = count_n_grams(train, 1)
bigram_counts = count_n_grams(train, 2)
trigram_counts = count_n_grams(train, 3)
quadgram_counts = count_n_grams(train, 4)
qintgram_counts = count_n_grams(train, 5)

n_gram_counts_list = [unigram_counts, bigram_counts, trigram_counts, quadgram_counts, qintgram_counts]

def update_suggestions(change):
    texte_actuel = change["new"]
    
    # On préprocess le texte
    texte = texte_actuel.lower()
    texte = texte.replace('\n', ' ')
    texte_without_space = re.sub(r'[^\w\s]', '', texte)
    texte_without_space = nltk.word_tokenize(texte_without_space) # Tableau des mots du texte sans les espaces
    texte_processed = replace_oov([texte_without_space], voc) # Tableau des mots du texte avec les mots inconnus remplacés par <unk>

    texte_splitted = texte.split(" ") # Tableau des mots du texte. texte_splitted[-1] contient un mot vide si le dernier caracère tapé est un espace

    # Calcul du prefixe et des tokens précédents 
    if texte_splitted[-1] == "": # Si le dernier caractère est un espace, on fixe le préfixe à "" et on prend les 4 derniers mots comme tokens 
        prefixe = ""
        previous_tokens = tuple(texte_processed[0][-4:]) # On prend les 4 derniers mots comme tokens
    else:
        prefixe = texte_without_space[-1] # On prend le dernier mot comme préfixe (qui est celui en cours d'écriture)
        if len(texte_processed[0]) >= 5: # Si le texte contient au moins 5 mots (celui en cours d'écriture + 4 mots précédents), on prend les 4 précédents comme tokens
            previous_tokens = tuple(texte_processed[0][-5:-1])
        else: # Sinon on prend tous les mots précédents à celui en cours d'écriture
            previous_tokens = tuple(texte_processed[0][:-1])
    
    # On récupère les suggestions
    if len(texte_processed) == 0:
        top_suggestions = []
    else:
        top_suggestions = get_suggestions(previous_tokens, n_gram_counts_list, voc, k=1.0, prefixe=prefixe)
        suggestions_label.value = "Suggestions: " + ", ".join(top_suggestions)

text_input.observe(update_suggestions, names="value")

display(text_input)
display(suggestions_label)

Text(value='', placeholder='Entrez votre text ici...')

Label(value='Suggestions: ')

<a name='5'></a>
## 5. Modèle de génération de phrases (30 points)

Dans cette partie vous allez construire un modèle de génération de phrases en utilisant les n-grammes.


#### Dans la cadre d'un modèle de génération de phrases, indiquez pourquoi la stratégie de suggestion des mots en 4. ne peut pas fonctionner ? (3 points)

Notre modèle d'auto-complétion suggère un mot à la fois basé sur le n-gramme précédent. Donc les propositions seront toujours identiques pour un même n-gramme précédent. (et notamment pour le premier mot où le n-gramme précédent est toujours \<s> et donc les suggestions du premier mot seront toujours les memes)  
  
Ainsi, ce modèle ne permet pas de générer des phrases car il propose toujours les memes mots pour un meme contexte.

<a name='5.1'></a>

### 5.1 Génération stochastique de mots (5 points)

Recodez la fonction suggest_word afin d'utiliser une suggestion stochastique. Autrement dit, au lieu de retourner le mot le plus probable, vous devrez générez le mot suivant selon sa probabilité.

Par exemple si le mot 'like' a la probabilité 0.25 d'être généré, alors il sera retourné 25% du temps.


In [77]:
def suggest_word_with_probs(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0):
    probabilities = estimate_probabilities(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k)

    random_word = np.random.choice(list(probabilities.keys()), p=list(probabilities.values()))
    probability = probabilities[random_word]

    return random_word, probability

<a name='5.2'></a>
### 5.2 Générations de phrases (10 points)

#### 5.2.1. Génération stochastique (4 points)
Complétez maintenant la fonction `generate_sentence` qui génère une phrase longue de n_words en appelant votre nouvelle fonction `suggest_words_with_probs`. La génération doit s'arrêter si le modèle génère un jeton de fin de phrase.

Il ne faut pas oublier d'initialiser les phrases à générer avec le bon nombre de jetons de début de phrase (`<s>`). Par exemple, s'il s'agit d'un modèle bigramme, il faudra initialiser la phrase à [`<s>`]. S'il s'agit d'un modèle trigramme, il faudra initialiser la phrase à [`<s>`, `<s>`]. Vous pouvez trouver la taille du contexte à l'aide de l'expression suivante `len(next(iter(n_gram_counts)))`.

Ensuite, il faudra passer à la fonction `suggest_word` les `n` derniers mots générés où `n` correspond à la taille du contexte.
Finalement, il faudra arrêter la génération si le jeton généré est le jeton de fin (`<e>`)

In [102]:
def generate_sentence(n_words, n_gram_counts, n_plus1_gram_counts, vocabulary, k=0.001):

    len_context = len(next(iter(n_gram_counts)))

    sentence = ["<s>"] * len_context
    sentence_finished = False

    while not sentence_finished:
        previous_n_gram = tuple(sentence[-len_context:])
        word, prob = suggest_word_with_probs(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k)
        sentence.append(word)
        if word == "<e>" or len(sentence) - len_context >= n_words:
            sentence_finished = True

    return " ".join(sentence[len_context:])


#### 5.2.2. Test sur des n-grammes (2 points)
Testez ensuite votre fonction avec des trigrammes et des 5-grammes.

In [103]:
unigram_counts = count_n_grams(train, 1)
bigram_counts = count_n_grams(train, 2)
trigram_counts = count_n_grams(train, 3)
quadrigram_counts = count_n_grams(train, 4)
quintigram_counts = count_n_grams(train, 5)


In [104]:
print("Phrase générée avec le modèle tri-gramme :")
print(generate_sentence(10, bigram_counts, trigram_counts, voc))
print()
print("Phrase générée avec le modèle 5-gramme :")
print(generate_sentence(10, quadrigram_counts, quintigram_counts, voc))


Phrase générée avec le modèle tri-gramme :
we want <e>

Phrase générée avec le modèle 5-gramme :
so i hope lane basics planning friday impressed fellows drawn


#### 5.2.3. Avec k=1.0, que se passe-t-il avec les phrases générées et quelle en est la raison principale ? Que pouvez-vous faire pour améliorer la situation? (2 points)

In [76]:
print("Phrase générée avec le modèle tri-gramme :")
print(generate_sentence(10, bigram_counts, trigram_counts, voc, k=1))
print()
print("Phrase générée avec le modèle 5-gramme :")
print(generate_sentence(10, quadrigram_counts, quintigram_counts, voc, k=1))


Phrase générée avec le modèle tri-gramme :
were lynch scene quite 4000 hostility can undermined system ludicrous

Phrase générée avec le modèle 5-gramme :
air campaigned engineer constitution cross paying guys projects creating email


Avec une valeur de k trop élevée, les phrases générees n'ont pas de sens. Cela est dû au lissage qui est trop important et qui fait que les probabilités de suite de mots n'ayant aucune corrélations sont trop élévées.
Pour améliorer la situation, on peut diminuer la valeur de k jusqu'à ce que les phrases générées soient plus cohérentes.

#### 5.2.4.  Quels sont les problèmes si la constante k a une valeur trop petite, voir 0?  (2 points)

In [40]:
print("Phrase générée avec le modèle tri-gramme :")
print(generate_sentence(10, bigram_counts, trigram_counts, voc, k=0.00000001))
print()
print("Phrase générée avec le modèle 5-gramme :")
print(generate_sentence(10, quadrigram_counts, quintigram_counts, voc, k=0.00000001))


Phrase générée avec le modèle tri-gramme :


more than one in four years its going to fight

Phrase générée avec le modèle 5-gramme :
general mike flynn <e>


Avec une valeur de k très faible, les phrases générées sont proches de celles du corpus d'entrainement. Certaines sont mêmes identiques (par exemple : we will be the big tent party <e>)
En effet, avec une valeur de k très faible, il n'y a quasiment aucun lissage et donc certaines suites de mots existant dans le corpus d'entrainement auront des probabilités élévées d'être générés.

<a name='5.3'></a>
### 5.3. Amélioration de la génération stochastique de mots (12 points)

#### 5.3.1. Amélioration stochastique

Comme vous avez pu l'observer, la génération stochastique, bien qu'elle soit efficace pour générer des phrases différentes, a tendance à ne pas générer des phrases toujours cohérentes. Proposez une amélioration de la méthode `suggest_word` que vous implémenterez dans la méthode `suggest_word_new` permettant de générer des phrases plus cohérentes.

##### a) Décrivez votre méthode dans la cellule suivante (3 points)

On propose d'introduire un paramètre t qui va permettre de jouer sur le caractère stochastique de la génération.  
Pour cela, on va calculer les probabilités de la même manière que précédemment, mais on va ensuite les modifier en les élévant à la puissance 1/t.  
  
Donc si t est proche de 1, les probabilités ne seront pas modifiées et la génération sera stochastique.  
Si t est plus faible, 1/t sera plus grand et cela aura pour effet d'augmenter les ecart entre les probabilités. Ainsi, les mots les plus probables seront encore plus probables et les mots les moins probables seront encore moins probables.  

On pensera à normaliser les probabilités après avoir appliqué la puissance 1/t pour que la somme des probabilités soit toujours égale à 1.  

Ainsi, on pourra jouer sur le caractère stochastique de la génération en faisant varier ce paramètre t pour garder une part de génération "aléatoire" tout en gardant la cohérence des phrases générées.

##### b) Implémentez la méthode proposée (5 points)

In [91]:
def suggest_word_new(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0, t=0.8):

    probabilities = estimate_probabilities(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k)

    # On applique la puissance 1/t aux probabilités
    probabilities = {word: prob**(1/t) for word, prob in probabilities.items()}
    # On normalise les probabilités
    sum_probabilities = sum(probabilities.values())
    probabilities = {word: prob/sum_probabilities for word, prob in probabilities.items()}

    # On tire un mot au hasard en fonction des probabilités
    random_word = np.random.choice(list(probabilities.keys()), p=list(probabilities.values()))
    probability = probabilities[random_word]

    return random_word, probability

#### 5.3.2. Génération améliorée (2 points)
Recodez maintenant la fonction `generate_sentence_new` pour appeler votre nouvelle méthode `suggest_word_new`.

In [89]:
def generate_sentence_new(n_words, n_gram_counts, n_plus1_gram_counts, vocabulary, k=0.001,  t = 0.8):

    len_context = len(next(iter(n_gram_counts)))

    sentence = ["<s>"] * len_context
    sentence_finished = False

    while not sentence_finished:
        previous_n_gram = tuple(sentence[-len_context:])
        word = suggest_word_new(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k, t)[0]
        sentence.append(word)
        if word == "<e>" or len(sentence) - len_context >= n_words:
            sentence_finished = True

    return " ".join(sentence[len_context:])


#### 5.3.3. Test sur des n-grammes (2 points)
Testez ensuite votre fonction avec des 3-grammes et des 5-grammes et validez que les phrases sont plus cohérentes.

In [106]:
print("Phrase générée avec le modèle tri-gramme :")
print(generate_sentence_new(10, bigram_counts, trigram_counts, voc, t=0.9))
print()
print("Phrase générée avec le modèle 5-gramme :")
print(generate_sentence_new(10, quadrigram_counts, quintigram_counts, voc, t=0.9))

Phrase générée avec le modèle tri-gramme :
do not have a lot of money and theyre smart

Phrase générée avec le modèle 5-gramme :
when i see cubans for trump census tackle 285000 establishment


## LIVRABLES:
Vous devez remettre sur Moodle, avant la date d'échéance, un zip contenant les fichiers suivants :

1-	Le code : Vous devez compléter le squelette inf8460_tp2.ipynb sous le nom   equipe_i_inf8460_TP2.ipynb (i = votre numéro d’équipe). Indiquez vos noms et matricules au début du notebook. Ce notebook doit contenir les fonctionnalités requises avec des commentaires appropriés. Le code doit être exécutable sans erreur et accompagné de commentaires appropriés de manière à expliquer les différentes fonctions. Les critères de qualité tels que la lisibilité du code et des commentaires sont importants. Tout votre code et vos résultats doivent être exécutables et reproductibles ;

2-	Un fichier pdf représentant votre notebook complètement exécuté sous format pdf.
Pour créer le fichier cliquez sur File > Download as > PDF via LaTeX (.pdf). Assurez-vous que le PDF est entièrement lisible.


## EVALUATION
Votre TP sera évalué selon les critères suivants :

1. Exécution correcte du code
2. Qualité du code (noms significatifs, structure, gestion d’exception, etc.) avec, entre autres, les recommandations suivantes:
    - Il ne devrait pas y avoir de duplication de code. Utilisez des fonctions pour garder votre code modulaire
    - Votre code devrait être optimisé: un code trop lent entraînera une perte de points
3. Lisibilité du code (Commentaires clairs et informatifs au besoin)
4. Performance/sortie attendue des modèles
5. Réponses correctes/sensées aux questions de réflexion ou d'analyse
